### Clinvar variants

Retrieval:
1. Downloaded form ClinVar FTP website: https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar_20230527.vcf.gz
   `/ibex/scratch/projects/c2102/databases/clinvar/2023_05/clinvar_20230527.vcf (decompressed)`

2. Split into chromosomes with bcftools
   `/ibex/scratch/projects/c2102/databases/clinvar/2023_05/clinvar_chr##.vcf`

Processing:
1. Annotated with VEP
   `/ibex/scratch/projects/c2102/databases/clinvar/2023_05/chr#/chr#_x##.vcf.vep`  

2. Filter out variants that contain `'not_specified'` or `'not_provided'` in the CLNDN field (phenotype/disease)

3. Select variants that contain `'missense_variant'` in the `'Consequence'` column (from VEP)

4. Select varinats that contain either `'Likely_pathogenic'` or `'Pathogenic'` in the `'CLNSIG'` column (from ClinVar)

5. Select variants where the gene provided by ClinVar (`GENEINFO` column) is equal to the gene in the VEP annotation (`SYMBOL` column)

6. Select variants where the RefSeq ID (RefSeq column) is equal to the transcript indicated in the `MANE_SELECT` or `MANE_PLUS_CLINICAL` columns (from VEP)

7. Obtained UniProt IDs for each variant, from two sources (sometimes there's more than 1 UniProt ID for a given transcript):  
   a) `UNIPROT_ISOFORM` column (from VEP)  
   b) UniProt database (`/ibex/projects/c2102/databases/uniprot/2023_02/current_release/knowledgebase/idmapping/by_organism/HUMAN_9606_idmapping.dat.gz`)

8. Get the AlphaFold model for each transcript, from two sources:  
   a) `MANE` dataset from the AlphaFold Database (`/ibex/scratch/projects/c2102/databases/alphafold/alphafold_mane`)  
   b) Regular AlphaFold Database (`/ibex/scratch/projects/c2102/databases/alphafold/alphafold_human_v4` )

   Checked that the sequence length and residue match.

9. Obtained intra-molecular contacts (`8 Angstroms` away, will change to 6)

10. Obtained catalytic sites for residue of interest and residues 8 Angstroms away (`variant_annotator.structure_features.catalyticSites`)

11. Annotated AlphaFold models with DSSP to calculate secondary structure and surface accessible area for every residue, and extracted the values for the variants.

12. Extracted pLDDT scores in a window of 5 residues centered around the residue of interest (proxy for disorder)

13. Annotated with FoldX  

In [1]:
import pandas as pd
import sys
import os
import numpy as np
from Bio.Data import IUPACData

In [2]:
protein_letters_3to1 = {value:key for key,value in IUPACData.protein_letters_1to3.items()}
protein_letters_1to3 = {key:value for key,value in IUPACData.protein_letters_1to3.items()}

In [3]:
pd.set_option('display.max_rows', 500)

In [4]:
pd.set_option('display.min_rows', 15)

In [3]:
clinvar_variants = pd.read_pickle('variants/clinvar_variants.pkl')

In [5]:
clinvar_variants.shape

(31252, 100)

In [7]:
clinvar_variants.columns.values.tolist()

['#CHROM',
 'POS',
 'ID',
 'REF',
 'ALT',
 'AF_ESP',
 'AF_EXAC',
 'AF_TGP',
 'ALLELEID',
 'CLNDN',
 'CLNDNINCL',
 'CLNDISDB',
 'CLNDISDBINCL',
 'CLNHGVS',
 'CLNREVSTAT',
 'CLNSIG',
 'CLNSIGCONF',
 'CLNSIGINCL',
 'CLNVC',
 'CLNVCSO',
 'CLNVI',
 'DBVARID',
 'MC',
 'ORIGIN',
 'RS',
 'Allele',
 'Consequence',
 'IMPACT',
 'SYMBOL',
 'Gene',
 'Feature_type',
 'Feature',
 'BIOTYPE',
 'EXON',
 'INTRON',
 'HGVSc',
 'HGVSp',
 'cDNA_position',
 'CDS_position',
 'Protein_position',
 'Amino_acids',
 'Codons',
 'Existing_variation',
 'DISTANCE',
 'STRAND',
 'VARIANT_CLASS',
 'SYMBOL_SOURCE',
 'HGNC_ID',
 'CANONICAL',
 'MANE_SELECT',
 'MANE_PLUS_CLINICAL',
 'ENSP',
 'SWISSPROT',
 'TREMBL',
 'UNIPARC',
 'UNIPROT_ISOFORM',
 'RefSeq',
 'SIFT',
 'PolyPhen',
 'DOMAINS',
 'CLIN_SIG',
 'SOMATIC',
 'PHENO',
 'Conservation',
 'BLOSUM62',
 'GENEINFO_NAME',
 'GENEINFO_ID',
 'RefSeq_noversion',
 'UniProt_IDs',
 'PDB_path',
 'Residue_position',
 'Residue',
 'DSSP_path',
 'intra_contacts',
 'is_catalytic',
 'pLDDT

In [ ]:
clinvar_variants['intra_contacts'].value_counts()

In [7]:
clinvar_variants['#CHROM'].value_counts()

2     3367
X     2953
1     2582
17    2291
19    1855
3     1774
12    1772
15    1564
11    1551
7     1537
16    1327
5     1198
9     1167
6     1056
14     957
10     929
4      743
20     638
8      631
13     626
18     453
21     264
Y       17
Name: #CHROM, dtype: int64

In [8]:
clinvar_variants['accessibility'].value_counts()

0      4972
1      1489
2       929
3       717
4       699
5       609
7       479
6       474
9       431
8       415
13      400
12      385
10      375
11      370
15      330
14      313
17      297
16      297
18      288
20      281
19      269
22      268
21      261
30      255
23      244
27      239
31      231
29      231
24      229
28      223
25      216
35      215
32      206
34      200
39      196
69      195
33      193
38      192
26      191
40      191
45      188
47      188
41      178
66      177
60      174
37      174
42      174
73      173
36      173
61      172
62      170
65      170
76      169
57      168
71      166
52      164
64      164
43      163
63      163
54      163
72      162
44      162
68      161
46      160
70      159
80      156
59      153
74      152
56      149
51      149
49      149
67      148
55      145
53      141
50      139
58      138
75      136
48      136
77      136
81      129
78      105
82      104
84      101
79  

In [9]:
clinvar_variants.loc[31245]

#CHROM                                                                    Y
POS                                                                 2787392
ID                                                                  1470948
REF                                                                       G
ALT                                                                       T
AF_ESP                                                                  NaN
AF_EXAC                                                                 NaN
AF_TGP                                                                  NaN
ALLELEID                                                            1395175
CLNDN                                                  46,XY_sex_reversal_1
CLNDNINCL                                                               NaN
CLNDISDB                  MONDO:MONDO:0020712,MedGen:C2748896,OMIM:40004...
CLNDISDBINCL                                                            NaN
CLNHGVS     

In [10]:
numeric_columns = clinvar_variants.select_dtypes(include=['int', 'float']).columns
print(numeric_columns)

Index(['POS', 'ID', 'HGVSc', 'HGVSp', 'Conservation', 'BLOSUM62',
       'Residue_position', 'pLDDT', 'accessibility', 'total energy',
       'Backbone Hbond', 'Sidechain Hbond', 'Van der Waals', 'Electrostatics',
       'Solvation Polar', 'Solvation Hydrophobic', 'Van der Waals clashes',
       'entropy sidechain', 'entropy mainchain', 'sloop_entropy',
       'mloop_entropy', 'cis_bond', 'torsional clash', 'backbone clash',
       'helix dipole', 'water bridge', 'disulfide', 'electrostatic kon',
       'partial covalent bonds', 'energy Ionisation', 'Entropy Complex'],
      dtype='object')


In [11]:
clinvar_variants['secondary_structure'].isnull().sum()

7481

In [12]:
clinvar_variants['CLNVC'].value_counts()

single_nucleotide_variant    31058
Indel                          176
Inversion                       18
Name: CLNVC, dtype: int64

In [54]:
clinvar_filtered = clinvar_variants[clinvar_variants['CLNVC']=='single_nucleotide_variant'].copy()

In [55]:
clinvar_filtered['CLNVC'].value_counts()

single_nucleotide_variant    31058
Name: CLNVC, dtype: int64

In [56]:
clinvar_filtered['Conservation'].isnull().sum()

1287

In [57]:
clinvar_filtered['AF_TGP'].isnull().sum()

30732

In [58]:
clinvar_filtered['AF_TGP'].value_counts()

0.0002     238
0.0004      36
0.0006      13
0.0008       6
0.001        5
0.00053      3
0.0024       3
0            3
0.0014       2
0.002        2
0.00026      2
0.0012       2
0.0018       1
0.00319      1
0.01358      1
0.00359      1
0.0016       1
0.28894      1
0.01478      1
0.45647      1
0.00459      1
0.03603      1
0.02396      1
Name: AF_TGP, dtype: int64

In [59]:
clinvar_filtered['CLNDN'].value_counts()

Early_infantile_epileptic_encephalopathy_with_suppression_bursts                                                                                                                                                                                                           422
Hypercholesterolemia,_familial,_1                                                                                                                                                                                                                                          367
Inborn_genetic_diseases                                                                                                                                                                                                                                                    354
Ehlers-Danlos_syndrome,_type_4                                                                                                                                                             

In [60]:
clinvar_filtered.shape

(31058, 100)

In [61]:
# Define the keywords to search for
keywords = ['heart', 'cardiovascular', 'cancer', 'carcinoma', 'sarcoma', 'leuchemia', 'lymphoma', 'myeloma']

# Add a new column based on the annotation
clinvar_filtered['DiseaseClass'] = clinvar_filtered['CLNDN'].apply(lambda x: 'heart disease' if any(keyword in str(x).lower() for keyword in keywords[:2]) else ('cancer' if 'cancer' in str(x).lower() else ('carcinoma' if 'carcinoma' in str(x).lower() else ('sarcoma' if 'sarcoma' in str(x).lower() else ('leuchemia' if 'leuchemia' in str(x).lower() else ('lymphoma' if 'lymphoma' in str(x).lower() else ('myeloma' if 'myeloma' in str(x).lower() else None)))))))

In [62]:
clinvar_filtered['DiseaseClass'].value_counts()

cancer           820
heart disease    420
carcinoma        414
myeloma           34
lymphoma          24
sarcoma            7
Name: DiseaseClass, dtype: int64

### add polarities & change in size & add number of contacts

In [63]:
clinvar_filtered[['WT', 'MUT']] = clinvar_filtered['Amino_acids'].str.split('/', expand=True)

In [64]:
# Convert one-letter code to three-letter code
clinvar_filtered['WT'] = clinvar_filtered['WT'].map(protein_letters_1to3)
clinvar_filtered['MUT'] = clinvar_filtered['MUT'].map(protein_letters_1to3)

In [65]:
clinvar_filtered.loc[31245]

#CHROM                                                                    Y
POS                                                                 2787392
ID                                                                  1470948
REF                                                                       G
ALT                                                                       T
AF_ESP                                                                  NaN
AF_EXAC                                                                 NaN
AF_TGP                                                                  NaN
ALLELEID                                                            1395175
CLNDN                                                  46,XY_sex_reversal_1
CLNDNINCL                                                               NaN
CLNDISDB                  MONDO:MONDO:0020712,MedGen:C2748896,OMIM:40004...
CLNDISDBINCL                                                            NaN
CLNHGVS     

In [66]:
aliphatic = [
    'Ala',
    'Ile',
    'Leu',
    'Met',
    'Val'
]

aromatic = [
    'Phe',
    'Trp',
    'Tyr'
]

hydrophobic = aliphatic + aromatic

positive = [
    'Arg',
    'His',
    'Lys'
]

negative = [
    'Asp',
    'Glu'
]

neutral = [
    'Asn',
    'Gln',
    'Thr',
    'Ser'
]

polar = positive + negative + neutral

special = [
    'Pro',
    'Gly',
    'Cys'
]

In [67]:
clinvar_filtered['aliphatic_assign_WT'] = clinvar_filtered.WT.isin(aliphatic).apply(
                            lambda x: 1 if x else 0)
features_vector = []
clinvar_filtered['aromatic_assign_WT'] = clinvar_filtered.WT.isin(aromatic).apply(
                            lambda x: 1 if x else 0)
clinvar_filtered['hydrophobic_assign_WT'] = clinvar_filtered.WT.isin(hydrophobic).apply(
                            lambda x: 1 if x else 0)
clinvar_filtered['positive_assign_WT'] = clinvar_filtered.WT.isin(positive).apply(
                            lambda x: 1 if x else 0)
clinvar_filtered['negative_assign_WT'] = clinvar_filtered.WT.isin(negative).apply(
                            lambda x: 1 if x else 0)
clinvar_filtered['neutral_assign_WT'] = clinvar_filtered.WT.isin(neutral).apply(
                            lambda x: 1 if x else 0)
clinvar_filtered['polar_assign_WT'] = clinvar_filtered.WT.isin(polar).apply(
                            lambda x: 1 if x else 0)
clinvar_filtered['special_assign_WT'] = clinvar_filtered.WT.isin(special).apply(
                            lambda x: 1 if x else 0)

In [68]:
clinvar_filtered['aliphatic_assign_MUT'] = clinvar_filtered.MUT.isin(aliphatic).apply(
                            lambda x: 1 if x else 0)
features_vector = []
clinvar_filtered['aromatic_assign_MUT'] = clinvar_filtered.MUT.isin(aromatic).apply(
                            lambda x: 1 if x else 0)
clinvar_filtered['hydrophobic_assign_MUT'] = clinvar_filtered.MUT.isin(hydrophobic).apply(
                            lambda x: 1 if x else 0)
clinvar_filtered['positive_assign_MUT'] = clinvar_filtered.MUT.isin(positive).apply(
                            lambda x: 1 if x else 0)
clinvar_filtered['negative_assign_MUT'] = clinvar_filtered.MUT.isin(negative).apply(
                            lambda x: 1 if x else 0)
clinvar_filtered['neutral_assign_MUT'] = clinvar_filtered.MUT.isin(neutral).apply(
                            lambda x: 1 if x else 0)
clinvar_filtered['polar_assign_MUT'] = clinvar_filtered.MUT.isin(polar).apply(
                            lambda x: 1 if x else 0)
clinvar_filtered['special_assign_MUT'] = clinvar_filtered.MUT.isin(special).apply(
                            lambda x: 1 if x else 0)

In [89]:
clinvar_filtered

,#CHROM,POS,ID,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,ALLELEID,CLNDN,...,polar_assign_WT,special_assign_WT,aliphatic_assign_MUT,aromatic_assign_MUT,hydrophobic_assign_MUT,positive_assign_MUT,negative_assign_MUT,neutral_assign_MUT,polar_assign_MUT,special_assign_MUT
0,1,197429459,1048144,G,C,NaN,NaN,NaN,1036051,Leber_congenital_amaurosis_8,...,0,1,0,0,0,0,0,1,1,0
1,1,197429466,1213987,T,G,NaN,NaN,NaN,1203980,Retinitis_pigmentosa_12,...,1,0,0,0,0,1,0,0,1,0
2,1,197429467,1963628,G,A,NaN,NaN,NaN,2017299,Retinitis_pigmentosa_12|Leber_congenital_amaurosis_8,...,0,1,0,0,0,1,0,0,1,0
3,1,197429468,2202910,G,C,NaN,NaN,NaN,1928560,Retinitis_pigmentosa_12|Leber_congenital_amaurosis_8,...,0,1,1,0,1,0,0,0,0,0
4,1,197429473,978993,G,T,NaN,NaN,NaN,967053,Autosomal_recessive_retinitis_pigmentosa,...,0,0,0,1,1,0,0,0,0,0
5,1,197429555,191357,G,A,NaN,NaN,NaN,189156,Retinitis_pigmentosa,...,0,1,0,1,1,0,0,0,0,0
6,1,197429581,813170,G,C,NaN,NaN,NaN,801275,Leber_congenital_amaurosis,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31051,Y,2787392,1470948,G,T,NaN,NaN,NaN,1395175,"46,XY_sex_reversal_1",...,1,0,0,1,1,0,0,0,0,0
31052,Y,2787401,9746,A,G,NaN,NaN,NaN,24785,"46,XY_sex_reversal_1",...,0,0,0,0,0,0,0,1,1,0


In [88]:
clinvar_filtered = clinvar_filtered.reset_index(drop=True)

In [90]:
len(clinvar_filtered)

31058

#### add changes in polarity

In [142]:
# Function to extract property names from column names
def extract_property(column_name):
    return column_name.split('_')[0]

clinvar_filtered['changes']=''
for i in range(0,len(clinvar_filtered)):
    for column in clinvar_filtered.columns:
        if column.endswith('_WT'):
            if clinvar_filtered[column].loc[i] == 1:
                prop = extract_property(column)
                clinvar_filtered['changes'].loc[i]=clinvar_filtered['changes'].loc[i]+prop+'_'
        elif column.endswith('_MUT'):
            if clinvar_filtered[column].loc[i] == 1:
                if clinvar_filtered['changes'].str.contains('to').iloc[i] == False:
                    prop = extract_property(column)
                    clinvar_filtered['changes'].loc[i]=clinvar_filtered['changes'].loc[i]+'to_'+prop+'_'
                else:
                    prop = extract_property(column)
                    clinvar_filtered['changes'].loc[i]=clinvar_filtered['changes'].loc[i]+prop+'_'
    clinvar_filtered['changes'].loc[i] = clinvar_filtered['changes'].str[:-1].loc[i]

/var/folders/56/l75lqrhd7m97zxtpbdw9rbbc0000gp/T/ipykernel_8537/498000398.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clinvar_filtered['changes'].loc[i]=clinvar_filtered['changes'].loc[i]+prop+'_'
/var/folders/56/l75lqrhd7m97zxtpbdw9rbbc0000gp/T/ipykernel_8537/498000398.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clinvar_filtered['changes'].loc[i]=clinvar_filtered['changes'].loc[i]+'to_'+prop+'_'
/var/folders/56/l75lqrhd7m97zxtpbdw9rbbc0000gp/T/ipykernel_8537/498000398.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentatio

In [143]:
clinvar_filtered

,#CHROM,POS,ID,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,ALLELEID,CLNDN,...,special_assign_WT,aliphatic_assign_MUT,aromatic_assign_MUT,hydrophobic_assign_MUT,positive_assign_MUT,negative_assign_MUT,neutral_assign_MUT,polar_assign_MUT,special_assign_MUT,changes
0,1,197429459,1048144,G,C,NaN,NaN,NaN,1036051,Leber_congenital_amaurosis_8,...,1,0,0,0,0,0,1,1,0,special_to_neutral_polar
1,1,197429466,1213987,T,G,NaN,NaN,NaN,1203980,Retinitis_pigmentosa_12,...,0,0,0,0,1,0,0,1,0,neutral_polar_to_positive_polar
2,1,197429467,1963628,G,A,NaN,NaN,NaN,2017299,Retinitis_pigmentosa_12|Leber_congenital_amaurosis_8,...,1,0,0,0,1,0,0,1,0,special_to_positive_polar
3,1,197429468,2202910,G,C,NaN,NaN,NaN,1928560,Retinitis_pigmentosa_12|Leber_congenital_amaurosis_8,...,1,1,0,1,0,0,0,0,0,special_to_aliphatic_hydrophobic
4,1,197429473,978993,G,T,NaN,NaN,NaN,967053,Autosomal_recessive_retinitis_pigmentosa,...,0,0,1,1,0,0,0,0,0,aliphatic_hydrophobic_to_aromatic_hydrophobic
5,1,197429555,191357,G,A,NaN,NaN,NaN,189156,Retinitis_pigmentosa,...,1,0,1,1,0,0,0,0,0,special_to_aromatic_hydrophobic
6,1,197429581,813170,G,C,NaN,NaN,NaN,801275,Leber_congenital_amaurosis,...,0,0,0,0,0,0,0,0,1,aliphatic_hydrophobic_to_special
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31051,Y,2787392,1470948,G,T,NaN,NaN,NaN,1395175,"46,XY_sex_reversal_1",...,0,0,1,1,0,0,0,0,0,neutral_polar_to_aromatic_hydrophobic
31052,Y,2787401,9746,A,G,NaN,NaN,NaN,24785,"46,XY_sex_reversal_1",...,0,0,0,0,0,0,1,1,0,aliphatic_hydrophobic_to_neutral_polar


In [144]:
clinvar_filtered['changes'].value_counts()

special_to_positive_polar                         2177
aliphatic_hydrophobic_to_aliphatic_hydrophobic    2099
aliphatic_hydrophobic_to_special                  1705
positive_polar_to_special                         1658
special_to_aliphatic_hydrophobic                  1605
aliphatic_hydrophobic_to_neutral_polar            1571
special_to_neutral_polar                          1538
positive_polar_to_neutral_polar                   1398
special_to_aromatic_hydrophobic                   1320
special_to_negative_polar                         1212
neutral_polar_to_positive_polar                   1195
neutral_polar_to_aliphatic_hydrophobic            1027
aromatic_hydrophobic_to_special                    894
positive_polar_to_positive_polar                   870
neutral_polar_to_special                           819
negative_polar_to_positive_polar                   818
aliphatic_hydrophobic_to_positive_polar            761
positive_polar_to_aromatic_hydrophobic             671
aromatic_h

### one-hot encoded and separation for columns: foldx & secondary structure (H,E,T,S,G,B,I) & is_catalytic & blosum62 (conservative & non-conservative)

In [158]:
clinvar_expanded = clinvar_filtered.copy()

In [159]:
one_hot = pd.get_dummies(clinvar_expanded['secondary_structure'], prefix='ss')

In [160]:
clinvar_expanded = pd.concat([clinvar_expanded, one_hot], axis=1)

In [165]:
clinvar_expanded['is_catalytic'] = clinvar_expanded['is_catalytic'].astype(int)

In [166]:
clinvar_expanded['is_catalytic']

0        0
1        0
2        0
3        0
4        0
5        0
6        0
        ..
31051    0
31052    0
31053    0
31054    0
31055    0
31056    0
31057    0
Name: is_catalytic, Length: 31058, dtype: int64

In [167]:
clinvar_expanded['is_catalytic'].value_counts()

0    30827
1      231
Name: is_catalytic, dtype: int64

In [176]:
clinvar_expanded['BLOSUM62'].isnull().sum()

0

In [171]:
# Example dataframe
df = pd.DataFrame({'BLOSUM62': [-2, 5, 0, -1, 11, -4, 3, -3, 7]})

# Create new columns and assign NaN values
df['conservative_changes'] = np.nan
df['nonconservative_changes'] = np.nan

# Set values based on conditions
df.loc[df['BLOSUM62'].between(0, 11), 'conservative_changes'] = df['BLOSUM62']
df.loc[df['BLOSUM62'].between(-4, -1), 'nonconservative_changes'] = df['BLOSUM62'].abs()

print(df)

   BLOSUM62  conservative_changes  nonconservative_changes
0        -2                   NaN                      2.0
1         5                   5.0                      NaN
2         0                   0.0                      NaN
3        -1                   NaN                      1.0
4        11                  11.0                      NaN
5        -4                   NaN                      4.0
6         3                   3.0                      NaN
7        -3                   NaN                      3.0
8         7                   7.0                      NaN


In [177]:
clinvar_expanded['BLOSUM62_nonNeg'] = clinvar_expanded['BLOSUM62'].apply(lambda x: max(0, x + 4))

In [180]:
clinvar_expanded.columns.to_list()

['#CHROM',
 'POS',
 'ID',
 'REF',
 'ALT',
 'AF_ESP',
 'AF_EXAC',
 'AF_TGP',
 'ALLELEID',
 'CLNDN',
 'CLNDNINCL',
 'CLNDISDB',
 'CLNDISDBINCL',
 'CLNHGVS',
 'CLNREVSTAT',
 'CLNSIG',
 'CLNSIGCONF',
 'CLNSIGINCL',
 'CLNVC',
 'CLNVCSO',
 'CLNVI',
 'DBVARID',
 'MC',
 'ORIGIN',
 'RS',
 'Allele',
 'Consequence',
 'IMPACT',
 'SYMBOL',
 'Gene',
 'Feature_type',
 'Feature',
 'BIOTYPE',
 'EXON',
 'INTRON',
 'HGVSc',
 'HGVSp',
 'cDNA_position',
 'CDS_position',
 'Protein_position',
 'Amino_acids',
 'Codons',
 'Existing_variation',
 'DISTANCE',
 'STRAND',
 'VARIANT_CLASS',
 'SYMBOL_SOURCE',
 'HGNC_ID',
 'CANONICAL',
 'MANE_SELECT',
 'MANE_PLUS_CLINICAL',
 'ENSP',
 'SWISSPROT',
 'TREMBL',
 'UNIPARC',
 'UNIPROT_ISOFORM',
 'RefSeq',
 'SIFT',
 'PolyPhen',
 'DOMAINS',
 'CLIN_SIG',
 'SOMATIC',
 'PHENO',
 'Conservation',
 'BLOSUM62',
 'GENEINFO_NAME',
 'GENEINFO_ID',
 'RefSeq_noversion',
 'UniProt_IDs',
 'PDB_path',
 'Residue_position',
 'Residue',
 'DSSP_path',
 'intra_contacts',
 'is_catalytic',
 'pLDDT

In [193]:
clinvar_expanded['DOMAINS'].isnull().sum()

2

In [194]:
clinvar_expanded['CANONICAL'].value_counts()

YES    30516
Name: CANONICAL, dtype: int64

In [196]:
clinvar_expanded['MANE_PLUS_CLINICAL'].value_counts()

NM_001371246.1    185
NM_014191.4       115
NM_001080463.2     50
NM_004992.4        43
NM_001099404.2     38
NM_003494.4        30
NM_001159702.3     17
NM_000828.5        12
NM_001006657.2      9
NM_030813.6         8
NM_015359.6         6
NM_001369387.1      6
NM_017890.5         5
NM_033056.4         5
NM_133379.5         4
NM_001365677.2      2
NM_005888.4         2
NM_001164507.2      2
NM_005709.4         1
NM_001329556.3      1
NM_001394928.1      1
Name: MANE_PLUS_CLINICAL, dtype: int64

In [202]:
clinvar_expanded.loc[31055]

#CHROM                                                                    Y
POS                                                                 2787515
ID                                                                   492908
REF                                                                       C
ALT                                                                       A
AF_ESP                                                                  NaN
AF_EXAC                                                                 NaN
AF_TGP                                                                  NaN
ALLELEID                                                             485861
CLNDN                                                  46,XY_sex_reversal_1
CLNDNINCL                                                               NaN
CLNDISDB                  MONDO:MONDO:0020712,MedGen:C2748896,OMIM:40004...
CLNDISDBINCL                                                            NaN
CLNHGVS     

In [189]:
clinvar_expanded['PDB_path'].loc[0]

PosixPath('/ibex/scratch/projects/c2102/databases/alphafold/alphafold_mane/AF-P82279-F1-model_v4.pdb')

In [181]:
!pip install ensembl-rest

  Preparing metadata (setup.py) ... done
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26098 sha256=539f8702b7784f392819d617557d0c807ae20821512310395085cc770f198e78
  Stored in directory: /Users/penaguka/Library/Caches/pip/wheels/ab/fa/1b/75d9a713279796785711bd0bad8334aaace560c0bd28830c8c
Successfully built intervaltree


In [204]:
import requests

def get_protein_length(transcript_id):
    if transcript_id in protein_length_cache:
        # If the protein length is already cached, return it
        return protein_length_cache[transcript_id]
    
    url = f"https://rest.ensembl.org/sequence/id/{transcript_id}?type=protein"
    headers = {"Content-Type": "application/json"}

    response = requests.get(url, headers=headers)

    if response.ok:
        protein_sequence = response.json()['seq']
        protein_length = len(protein_sequence)
        return protein_length
    else:
        print(f"Error retrieving protein length of {transcript_id}.")
        # Handle error if transcript ID is invalid or protein sequence is not available
        return None

# Example usage
transcript_id = "ENST00000383070"  # Replace with your transcript ID
length = get_protein_length(transcript_id)

if length is not None:
    print(f"Protein Length: {length}")
else:
    print("Error retrieving protein length.")

Protein Length: 204


In [205]:
clinvar_expanded['prot_length'] = clinvar_expanded['Feature'].apply(get_protein_length)

ConnectionError: HTTPSConnectionPool(host='rest.ensembl.org', port=443): Max retries exceeded with url: /sequence/id/ENST00000296589?type=protein (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fe5d7f7f520>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

### gnomAD variants

In [7]:
gnomad_variants = pd.read_pickle('variants/gnomad_variants.pkl')

In [8]:
gnomad_variants

,#CHROM,POS,ID,REF,ALT,AC,AF,AN,QD,variant_type,...,cis_bond,torsional clash,backbone clash,helix dipole,water bridge,disulfide,electrostatic kon,partial covalent bonds,energy Ionisation,Entropy Complex
0,chr1,11706367,rs11121804,C,T,39444,0.158732,248494,13.47,mixed,...,0.0,-0.018940,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0
1,chr1,11790870,rs2274976,C,T,13949,0.0555184,251250,14.18,snv,...,0.0,-0.165854,-0.304761,3.235800e-01,0.0,0.0,0.0,0.0,0.000000e+00,0.0
2,chr1,11824498,rs198400,A,G,250947,0.999399,251098,33.49,snv,...,0.0,-0.140793,-1.049150,-7.725940e-03,0.0,0.0,0.0,0.0,0.000000e+00,0.0
3,chr1,11847591,rs5063,C,T,14048,0.0558586,251492,13.96,snv,...,0.0,-0.104118,0.236702,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0
4,chr1,11949899,rs7551175,G,A,45490,0.180978,251356,14.94,snv,...,0.0,0.006133,0.028216,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15936,chrX,72140250,rs7061150,C,T,12196,0.0674621,180783,19.16,snv,...,0.0,0.031276,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0
15937,chrY,6246522,rs201475959,A,G,1025,0.0780952,13125,30.14,snv,...,0.0,-0.000050,-0.708370,-4.440890e-16,0.0,0.0,0.0,0.0,0.000000e+00,0.0
15938,chrY,12720687,rs7067496,G,T,6033,0.0894759,67426,32.95,snv,...,0.0,-0.002779,-0.131799,3.340370e-01,0.0,0.0,0.0,0.0,-8.881780e-16,0.0
15939,chrY,22180292,rs1478920645,G,T,1,0.142857,7,15.53,snv,...,0.0,0.002249,0.067966,-2.492200e-01,0.0,0.0,0.0,0.0,-1.996460e-03,0.0


In [10]:
gnomad_variants.columns.values.tolist()

['#CHROM',
 'POS',
 'ID',
 'REF',
 'ALT',
 'AC',
 'AF',
 'AN',
 'QD',
 'variant_type',
 'vep',
 'flags',
 'Allele',
 'Consequence',
 'IMPACT',
 'SYMBOL',
 'Gene',
 'Feature_type',
 'Feature',
 'BIOTYPE',
 'EXON',
 'INTRON',
 'HGVSc',
 'HGVSp',
 'cDNA_position',
 'CDS_position',
 'Protein_position',
 'Amino_acids',
 'Codons',
 'Existing_variation',
 'DISTANCE',
 'STRAND',
 'VARIANT_CLASS',
 'SYMBOL_SOURCE',
 'HGNC_ID',
 'CANONICAL',
 'MANE_SELECT',
 'MANE_PLUS_CLINICAL',
 'ENSP',
 'SWISSPROT',
 'TREMBL',
 'UNIPARC',
 'UNIPROT_ISOFORM',
 'RefSeq',
 'SIFT',
 'PolyPhen',
 'DOMAINS',
 'CLIN_SIG',
 'SOMATIC',
 'PHENO',
 'Conservation',
 'BLOSUM62',
 'RefSeq_noversion',
 'UniProt_IDs',
 'PDB_path',
 'Residue_position',
 'Residue',
 'DSSP_path',
 'intra_contacts',
 'is_catalytic',
 'pLDDT',
 'secondary_structure',
 'accessibility',
 'total energy',
 'Backbone Hbond',
 'Sidechain Hbond',
 'Van der Waals',
 'Electrostatics',
 'Solvation Polar',
 'Solvation Hydrophobic',
 'Van der Waals clashes',